## Test Anthropic Agent SDK

In [2]:
import asyncio
from claude_agent_sdk import query, ClaudeAgentOptions, AssistantMessage, ResultMessage

Testing `query` method

In [5]:
options = ClaudeAgentOptions(
    allowed_tools=[
        "Read",
        "Glob", 
        "Grep",
        "WebSearch",
        "WebFetch"
    ], #"Edit",
    #permission_mode="acceptEdits",
    system_prompt="You are a helpful AI assistant that is very efficient with words.",
    model="claude-sonnet-4-5",
    max_thinking_tokens=15_000,
    #user="sirius-black"
)

In [7]:
async for message in query(
    prompt="What files are in this directory?",
    options=options
):
    # Print human-readable output
    if isinstance(message, AssistantMessage):
        for block in message.content:
            if hasattr(block, "text"):
                print(block.text)              # Claude's reasoning
            elif hasattr(block, "name"):
                print(f"Tool: {block.name}")   # Tool being called
    elif isinstance(message, ResultMessage):
        print(f"Done: {message.subtype}")      # Final result

I'll list the files in the current directory for you.
Tool: Bash
This directory contains:

- **test-agents-sdk.ipynb** - An empty Jupyter notebook (0 bytes)
- **test-connections.ipynb** - A Jupyter notebook (16.9 KB)

Both are Jupyter notebook files (.ipynb).
Done: success


In [8]:
message

ResultMessage(subtype='success', duration_ms=11225, duration_api_ms=18733, is_error=False, num_turns=2, session_id='ef7acae2-e50c-4082-bc43-9425432094b8', total_cost_usd=0.0736793, usage={'input_tokens': 23, 'cache_creation_input_tokens': 15582, 'cache_read_input_tokens': 15276, 'output_tokens': 264, 'server_tool_use': {'web_search_requests': 0, 'web_fetch_requests': 0}, 'service_tier': 'standard', 'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 15582}}, result='This directory contains:\n\n- **test-agents-sdk.ipynb** - An empty Jupyter notebook (0 bytes)\n- **test-connections.ipynb** - A Jupyter notebook (16.9 KB)\n\nBoth are Jupyter notebook files (.ipynb).', structured_output=None)